 # Comprehensive Data Cleaning Pipeline for Bat vs Rat Research
============================================================

Investigation A: Do bats perceive rats not just as competitors for food but also as potential predators?

This script provides a thorough cleaning pipeline for both datasets used in the 
bat vs rat behavioral research specifically tailored for Investigation A.

Key Focus Areas:
- Avoidance behavior indicators (bat_landing_to_food delay times)
- Vigilance measures (risk-taking vs risk-avoidance behaviors)
- Temporal patterns in bat-rat interactions
- Food competition dynamics

Author: Amrit Niure and the HIT140 Team
Date: September 2025
Project: HIT140 Assessment 2 - Investigation A

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [42]:
# Load datasets
print("Loading datasets...")

df1 = pd.read_csv("raw_data/dataset1.csv")
df2 = pd.read_csv("raw_data/dataset2.csv")

print(f"Dataset 1 shape: {df1.shape}")
print(f"Dataset 2 shape: {df2.shape}")
print("\n" + "="*50)


Loading datasets...
Dataset 1 shape: (907, 12)
Dataset 2 shape: (2123, 7)



In [43]:
# Inspect Dataset 1 quality
print("\n=== DATA QUALITY INSPECTION: DATASET 1 ===")

# Basic info
print(f"Shape: {df1.shape}")
print(f"Columns: {list(df1.columns)}")

# Data types
print("\nData Types:")
print(df1.dtypes)

# Missing values
print("\nMissing Values:")
missing = df1.isnull().sum()
missing_pct = (missing / len(df1)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

# Duplicates
duplicates = df1.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Basic statistics for numeric columns
print("\nNumeric Column Statistics:")
numeric_cols = df1.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(df1[numeric_cols].describe())

print("\n" + "="*50)



=== DATA QUALITY INSPECTION: DATASET 1 ===
Shape: (907, 12)
Columns: ['start_time', 'bat_landing_to_food', 'habit', 'rat_period_start', 'rat_period_end', 'seconds_after_rat_arrival', 'risk', 'reward', 'month', 'sunset_time', 'hours_after_sunset', 'season']

Data Types:
start_time                    object
bat_landing_to_food          float64
habit                         object
rat_period_start              object
rat_period_end                object
seconds_after_rat_arrival      int64
risk                           int64
reward                         int64
month                          int64
sunset_time                   object
hours_after_sunset           float64
season                         int64
dtype: object

Missing Values:
       Missing Count  Missing %
habit             41   4.520397

Duplicate rows: 1

Numeric Column Statistics:
       bat_landing_to_food  seconds_after_rat_arrival        risk      reward  \
count           907.000000                 907.000000  907.000

In [44]:
# Inspect Dataset 2 quality
print("\n=== DATA QUALITY INSPECTION: DATASET 2 ===")

# Basic info
print(f"Shape: {df2.shape}")
print(f"Columns: {list(df2.columns)}")

# Data types
print("\nData Types:")
print(df2.dtypes)

# Missing values
print("\nMissing Values:")
missing = df2.isnull().sum()
missing_pct = (missing / len(df2)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

# Duplicates
duplicates = df2.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Basic statistics for numeric columns
print("\nNumeric Column Statistics:")
numeric_cols = df2.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(df2[numeric_cols].describe())

print("\n" + "="*50)



=== DATA QUALITY INSPECTION: DATASET 2 ===
Shape: (2123, 7)
Columns: ['time', 'month', 'hours_after_sunset', 'bat_landing_number', 'food_availability', 'rat_minutes', 'rat_arrival_number']

Data Types:
time                   object
month                   int64
hours_after_sunset    float64
bat_landing_number      int64
food_availability     float64
rat_minutes           float64
rat_arrival_number      int64
dtype: object

Missing Values:
Empty DataFrame
Columns: [Missing Count, Missing %]
Index: []

Duplicate rows: 0

Numeric Column Statistics:
             month  hours_after_sunset  bat_landing_number  food_availability  \
count  2123.000000         2123.000000         2123.000000        2123.000000   
mean      3.083844            5.265426           32.083373           2.445874   
std       1.642261            4.076188           25.614431           1.218353   
min       0.000000           -2.000000            0.000000           0.000000   
25%       2.000000            2.000000    

In [45]:
# Clean Dataset 1
print("\n=== CLEANING DATASET 1 (BAT BEHAVIOR DATA) ===")
df1_clean = df1.copy()

# 1. Handle datetime columns - CRITICAL for temporal analysis
print("1. Converting datetime columns...")
date_cols = ["start_time", "rat_period_start", "rat_period_end", "sunset_time"]

for col in date_cols:
    if col in df1_clean.columns:
        df1_clean[col] = pd.to_datetime(df1_clean[col], dayfirst=True, errors="coerce")
        print(f"   Converted {col} to datetime")



=== CLEANING DATASET 1 (BAT BEHAVIOR DATA) ===
1. Converting datetime columns...
   Converted start_time to datetime
   Converted rat_period_start to datetime
   Converted rat_period_end to datetime
   Converted sunset_time to datetime


In [46]:
# 2. Clean AVOIDANCE BEHAVIOR indicator (bat_landing_to_food)
print("2. Processing avoidance behavior indicators...")

if 'bat_landing_to_food' in df1_clean.columns:
    # This is KEY for Investigation A - time delay indicates avoidance/vigilance
    original_count = len(df1_clean)

    # Remove clearly erroneous values (negative times don't make sense)
    df1_clean = df1_clean[df1_clean['bat_landing_to_food'] >= 0]
    print(f"   Removed {original_count - len(df1_clean)} records with negative landing-to-food times")

    # Create avoidance behavior categories based on delay time
    # Longer delays = more avoidance/vigilance behavior
    df1_clean['avoidance_level'] = pd.cut(
        df1_clean['bat_landing_to_food'],
        bins=[0, 1, 5, 15, float('inf')],
        labels=['immediate', 'quick', 'delayed', 'highly_delayed'],
        include_lowest=True
    )

    # Binary indicator for high avoidance (>5 seconds delay)
    df1_clean['high_avoidance'] = (df1_clean['bat_landing_to_food'] > 5).astype(int)

    print("   Created avoidance behavior categories")


2. Processing avoidance behavior indicators...
   Removed 0 records with negative landing-to-food times
   Created avoidance behavior categories


In [47]:
# 3. Clean and enhance HABIT column for behavior analysis
print("3. Processing bat behavior patterns...")

# Handle missing and standardize
df1_clean['habit'] = df1_clean['habit'].fillna("unknown")
df1_clean['habit'] = df1_clean['habit'].astype(str).str.strip().str.lower()
df1_clean['habit'] = df1_clean['habit'].replace('', 'unknown')
df1_clean['habit'] = df1_clean['habit'].replace('nan', 'unknown')

# Remove coordinate-like anomalies
coordinate_pattern = r'\d+\.\d+,\d+\.\d+,\d+\.\d+,\d+\.\d+'
df1_clean.loc[df1_clean['habit'].str.contains(coordinate_pattern, na=False), 'habit'] = 'unknown'

# Create PREDATOR PERCEPTION indicators
def analyze_predator_perception(habit_value):
    """Analyze behavior for predator perception indicators."""
    if pd.isna(habit_value) or habit_value == 'unknown':
        return {'rat_present': 0, 'behavior_type': 'unknown', 'vigilance_level': 'unknown'}
    
    v = str(habit_value).lower().strip()
    
    # Rat presence detection
    rat_present = 1 if "rat" in v else 0
    
    # Behavior classification
    if "fight" in v or "bat_fight" in v:
        behavior_type = "aggressive"  # Direct confrontation
        vigilance_level = "high"
    elif "gaze" in v:
        behavior_type = "vigilant"    # Monitoring behavior
        vigilance_level = "high"
    elif "pick" in v:
        behavior_type = "feeding"     # Food acquisition
        vigilance_level = "medium" if rat_present else "low"
    elif "fast" in v:
        behavior_type = "quick_feeding"  # Rapid food acquisition
        vigilance_level = "medium"
    else:
        behavior_type = "other"
        vigilance_level = "unknown"
    
    return {
        'rat_present': rat_present,
        'behavior_type': behavior_type,
        'vigilance_level': vigilance_level
    }

# Apply predator perception analysis
behavior_analysis = df1_clean['habit'].apply(analyze_predator_perception)
df1_clean['rat_present'] = [analysis['rat_present'] for analysis in behavior_analysis]
df1_clean['behavior_type'] = [analysis['behavior_type'] for analysis in behavior_analysis]
df1_clean['vigilance_level'] = [analysis['vigilance_level'] for analysis in behavior_analysis]


3. Processing bat behavior patterns...


In [48]:
# 4. Process RISK and REWARD indicators - Core to Investigation A
print("4. Processing risk-reward behavioral indicators...")

if 'risk' in df1_clean.columns:
    df1_clean['risk'] = df1_clean['risk'].fillna(0).astype(int).clip(0, 1)
    # risk=1 indicates risk-taking behavior despite rat presence

if 'reward' in df1_clean.columns:
    df1_clean['reward'] = df1_clean['reward'].fillna(0).astype(int).clip(0, 1)
    # reward=1 indicates successful food acquisition

# Create predator-avoidance success indicators
if 'risk' in df1_clean.columns and 'reward' in df1_clean.columns:
    # Successful avoidance = got reward without taking risk
    df1_clean['successful_avoidance'] = ((df1_clean['reward'] == 1) & (df1_clean['risk'] == 0)).astype(int)
    
    # Risk-taking success = got reward despite taking risk
    df1_clean['risk_taking_success'] = ((df1_clean['reward'] == 1) & (df1_clean['risk'] == 1)).astype(int)


4. Processing risk-reward behavioral indicators...


In [49]:
# 5. Temporal analysis for Investigation A
print("5. Creating temporal features for predator perception analysis...")

if 'start_time' in df1_clean.columns:
    df1_clean['hour'] = df1_clean['start_time'].dt.hour
    df1_clean['day_of_week'] = df1_clean['start_time'].dt.dayofweek
    df1_clean['is_night'] = ((df1_clean['hour'] >= 18) | (df1_clean['hour'] <= 6)).astype(int)
    
    # Peak activity periods when predator encounters are more likely
    df1_clean['peak_activity'] = ((df1_clean['hour'] >= 19) & (df1_clean['hour'] <= 23)).astype(int)


5. Creating temporal features for predator perception analysis...


In [50]:
# 6. Rat encounter timing analysis
print("6. Analyzing rat encounter timing...")

if 'seconds_after_rat_arrival' in df1_clean.columns:
    # Time since rat arrival - key for predator perception
    df1_clean['seconds_after_rat_arrival'] = pd.to_numeric(
        df1_clean['seconds_after_rat_arrival'], errors='coerce'
    ).fillna(0)
    
    # Create encounter timing categories
    df1_clean['encounter_timing'] = pd.cut(
        df1_clean['seconds_after_rat_arrival'],
        bins=[0, 30, 120, 300, float('inf')],
        labels=['immediate', 'short_delay', 'medium_delay', 'long_delay'],
        include_lowest=True
    )
    
    # Quick encounter indicator (within 30 seconds of rat arrival)
    df1_clean['quick_encounter'] = (df1_clean['seconds_after_rat_arrival'] <= 30).astype(int)


6. Analyzing rat encounter timing...


In [51]:
# 7. Calculate rat presence duration for each bat landing
print("7. Calculating predator exposure duration...")

if all(col in df1_clean.columns for col in ['rat_period_start', 'rat_period_end']):
    # Calculate how long rats were present
    df1_clean['rat_presence_duration'] = (
        df1_clean['rat_period_end'] - df1_clean['rat_period_start']
    ).dt.total_seconds().fillna(0)
    
    # Long predator exposure indicator
    df1_clean['long_predator_exposure'] = (
        df1_clean['rat_presence_duration'] > 300
    ).astype(int)  # >5 minutes


7. Calculating predator exposure duration...


In [52]:
# 8. Create Investigation A specific composite indicators
print("8. Creating Investigation A composite indicators...")

# Predator avoidance score (higher = more avoidance behavior)
avoidance_factors = []

if 'high_avoidance' in df1_clean.columns:
    avoidance_factors.append(df1_clean['high_avoidance'])
    
if 'risk' in df1_clean.columns:
    avoidance_factors.append(1 - df1_clean['risk'])  # inverse of risk-taking
    
if 'vigilance_level' in df1_clean.columns:
    vigilance_score = df1_clean['vigilance_level'].map({'low': 0, 'medium': 1, 'high': 2, 'unknown': 0})
    avoidance_factors.append(vigilance_score / 2)  # normalize to 0-1

if avoidance_factors:
    df1_clean['predator_avoidance_score'] = np.mean(avoidance_factors, axis=0)

# Competition vs predation indicator
# High competition: rats present, feeding behavior, reward obtained
# High predation fear: rats present, delayed approach, no reward or high vigilance
if all(col in df1_clean.columns for col in ['rat_present', 'behavior_type', 'reward']):
    def classify_interaction_type(row):
        if row['rat_present'] == 0:
            return 'no_rat'
        elif row['behavior_type'] in ['feeding', 'quick_feeding'] and row['reward'] == 1:
            return 'competition'
        elif row['behavior_type'] in ['vigilant', 'aggressive'] or row['reward'] == 0:
            return 'predation_fear'
        else:
            return 'uncertain'
    
    df1_clean['interaction_type'] = df1_clean.apply(classify_interaction_type, axis=1)

# Summary
print(f"Dataset 1 cleaning complete. Final shape: {df1_clean.shape}")
print("Key Investigation A variables created:")
if 'predator_avoidance_score' in df1_clean.columns:
    print(f"   - Predator avoidance score (mean: {df1_clean['predator_avoidance_score'].mean():.3f})")
if 'interaction_type' in df1_clean.columns:
    print(f"   - Interaction types: {df1_clean['interaction_type'].value_counts().to_dict()}")


8. Creating Investigation A composite indicators...
Dataset 1 cleaning complete. Final shape: (907, 29)
Key Investigation A variables created:
   - Predator avoidance score (mean: 0.369)
   - Interaction types: {'no_rat': 596, 'predation_fear': 271, 'competition': 40}


In [53]:
# Clean Dataset 2
print("\n=== CLEANING DATASET 2 (TIME SERIES OBSERVATION DATA) ===")
df2_clean = df2.copy()

# 1. Handle datetime column
print("1. Converting datetime column...")
if 'time' in df2_clean.columns:
    df2_clean['time'] = pd.to_datetime(df2_clean['time'], dayfirst=True, errors="coerce")
    print("   Converted time to datetime")



=== CLEANING DATASET 2 (TIME SERIES OBSERVATION DATA) ===
1. Converting datetime column...
   Converted time to datetime


In [54]:
# 2. Clean and validate core behavioral metrics
print("2. Processing core behavioral metrics for Investigation A...")

# Key metrics for predator perception analysis
behavioral_cols = ['bat_landing_number', 'food_availability', 'rat_minutes', 'rat_arrival_number']

for col in behavioral_cols:
    if col in df2_clean.columns:
        # Handle negative values (shouldn't exist)
        negative_count = (df2_clean[col] < 0).sum()
        if negative_count > 0:
            print(f"   Found {negative_count} negative values in {col}, setting to 0")
            df2_clean[col] = df2_clean[col].clip(lower=0)
        
        # Fill missing values appropriately
        df2_clean[col] = df2_clean[col].fillna(0)


2. Processing core behavioral metrics for Investigation A...


In [55]:
# 3. Create PREDATOR PRESSURE indicators
print("3. Creating predator pressure indicators...")

if 'rat_minutes' in df2_clean.columns:
    df2_clean['rat_present'] = (df2_clean['rat_minutes'] > 0).astype(int)
    
    # Categorize predator pressure intensity
    df2_clean['predator_pressure'] = pd.cut(
        df2_clean['rat_minutes'],
        bins=[0, 0.1, 5, 15, float('inf')],
        labels=['none', 'minimal', 'moderate', 'high'],
        include_lowest=True
    )
    
    # High predator pressure indicator (>15 minutes of rat presence)
    df2_clean['high_predator_pressure'] = (df2_clean['rat_minutes'] > 15).astype(int)

if 'rat_arrival_number' in df2_clean.columns:
    df2_clean['multiple_rat_arrivals'] = (df2_clean['rat_arrival_number'] > 1).astype(int)
    df2_clean['frequent_rat_activity'] = (df2_clean['rat_arrival_number'] >= 3).astype(int)

# 4. Analyze BAT RESPONSE patterns to predator presence
print("4. Analyzing bat response patterns...")

if 'bat_landing_number' in df2_clean.columns:
    # Create bat activity categories
    df2_clean['bat_activity_level'] = pd.cut(
        df2_clean['bat_landing_number'],
        bins=[0, 10, 30, 50, float('inf')],
        labels=['low', 'moderate', 'high', 'very_high'],
        include_lowest=True
    )
    
    # High bat activity indicator
    df2_clean['high_bat_activity'] = (df2_clean['bat_landing_number'] > 50).astype(int)


3. Creating predator pressure indicators...
4. Analyzing bat response patterns...


In [56]:
# 5. FOOD COMPETITION analysis
print("5. Analyzing food competition dynamics...")

if 'food_availability' in df2_clean.columns:
    # Standardize food availability (should be 0-4 scale)
    df2_clean['food_availability'] = df2_clean['food_availability'].clip(0, 4)
    
    # Create food scarcity categories
    df2_clean['food_level'] = pd.cut(
        df2_clean['food_availability'],
        bins=[0, 1, 2, 3, 4],
        labels=['scarce', 'low', 'moderate', 'abundant'],
        include_lowest=True
    )
    
    # Food scarcity indicator
    df2_clean['food_scarce'] = (df2_clean['food_availability'] <= 1).astype(int)


5. Analyzing food competition dynamics...


In [57]:
# 6. Calculate COMPETITION vs PREDATION indicators
print("6. Creating competition vs predation indicators...")

# Bat efficiency when rats present vs absent
if all(col in df2_clean.columns for col in ['bat_landing_number', 'food_availability', 'rat_present']):
    # Avoid division by zero
    df2_clean['bat_efficiency'] = np.where(
        df2_clean['food_availability'] > 0,
        df2_clean['bat_landing_number'] / df2_clean['food_availability'],
        df2_clean['bat_landing_number']
    )
    
    # Initialize reduced efficiency indicator
    df2_clean['reduced_efficiency_with_rats'] = 0
    
    if df2_clean['rat_present'].sum() > 0:  # If there are periods with rats
        # Calculate efficiency when rats absent vs present
        no_rat_efficiency = df2_clean[df2_clean['rat_present'] == 0]['bat_efficiency'].mean()
        rat_efficiency = df2_clean[df2_clean['rat_present'] == 1]['bat_efficiency'].mean()
        
        if no_rat_efficiency > 0:
            efficiency_reduction = (no_rat_efficiency - rat_efficiency) / no_rat_efficiency
            df2_clean.loc[df2_clean['rat_present'] == 1, 'reduced_efficiency_with_rats'] = max(0, efficiency_reduction)


6. Creating competition vs predation indicators...


In [58]:
# 7. Temporal analysis for Investigation A
print("7. Creating temporal features...")

if 'time' in df2_clean.columns:
    df2_clean['hour'] = df2_clean['time'].dt.hour
    df2_clean['day_of_week'] = df2_clean['time'].dt.dayofweek
    df2_clean['is_night'] = ((df2_clean['hour'] >= 18) | (df2_clean['hour'] <= 6)).astype(int)
    df2_clean['date'] = df2_clean['time'].dt.date
    
    # Peak foraging times when predator encounters are more likely
    df2_clean['peak_foraging_time'] = ((df2_clean['hour'] >= 19) & (df2_clean['hour'] <= 23)).astype(int)


7. Creating temporal features...


In [59]:
# 8. PREDATOR AVOIDANCE patterns
print("8. Analyzing predator avoidance patterns...")

# Periods with rats but low bat activity = potential avoidance
if all(col in df2_clean.columns for col in ['rat_present', 'bat_landing_number']):
    median_bat_activity = df2_clean['bat_landing_number'].median()
    
    df2_clean['potential_avoidance'] = (
        (df2_clean['rat_present'] == 1) &
        (df2_clean['bat_landing_number'] < median_bat_activity)
    ).astype(int)
    
    # High avoidance: rats present but very low bat activity
    low_activity_threshold = df2_clean['bat_landing_number'].quantile(0.25)
    df2_clean['strong_avoidance'] = (
        (df2_clean['rat_present'] == 1) &
        (df2_clean['bat_landing_number'] <= low_activity_threshold)
    ).astype(int)


8. Analyzing predator avoidance patterns...


In [60]:
# 9. Create Investigation A summary metrics
print("9. Creating Investigation A summary indicators...")

# Predator-prey interaction intensity score
interaction_factors = []

if 'predator_pressure' in df2_clean.columns:
    pressure_score = df2_clean['predator_pressure'].map({'none': 0, 'minimal': 1, 'moderate': 2, 'high': 3})
    pressure_score = pd.to_numeric(pressure_score, errors='coerce').fillna(0)  # Convert to numeric
    interaction_factors.append(pressure_score / 3)  # normalize

if 'bat_activity_level' in df2_clean.columns:
    activity_score = df2_clean['bat_activity_level'].map({'low': 0, 'moderate': 1, 'high': 2, 'very_high': 3})
    activity_score = pd.to_numeric(activity_score, errors='coerce').fillna(0)  # Convert to numeric
    interaction_factors.append(activity_score / 3)  # normalize

if interaction_factors:
    df2_clean['interaction_intensity'] = np.mean(interaction_factors, axis=0)

# Competitive exclusion indicator
# High when: high rat presence but low bat activity AND low food
if all(col in df2_clean.columns for col in ['rat_present', 'bat_landing_number', 'food_availability']):
    df2_clean['competitive_exclusion'] = (
        (df2_clean['rat_present'] == 1) &
        (df2_clean['bat_landing_number'] < df2_clean['bat_landing_number'].median()) &
        (df2_clean['food_availability'] <= 2)
    ).astype(int)





9. Creating Investigation A summary indicators...


In [61]:
# 10. Outlier detection for key variables
print("10. Detecting outliers in key variables...")

outlier_cols = ['bat_landing_number', 'rat_minutes']
for col in outlier_cols:
    if col in df2_clean.columns:
        Q1 = df2_clean[col].quantile(0.25)
        Q3 = df2_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = (df2_clean[col] < lower_bound) | (df2_clean[col] > upper_bound)
        df2_clean[f'{col}_outlier'] = outliers
        print(f"   Found {outliers.sum()} outliers in {col}")


10. Detecting outliers in key variables...
   Found 24 outliers in bat_landing_number
   Found 511 outliers in rat_minutes


In [62]:
# Summary
print(f"Dataset 2 cleaning complete. Final shape: {df2_clean.shape}")
print("Key Investigation A variables created:")
if 'predator_pressure' in df2_clean.columns:
    print(f"   - Predator pressure levels: {df2_clean['predator_pressure'].value_counts().to_dict()}")
if 'potential_avoidance' in df2_clean.columns:
    print(f"   - Potential avoidance periods: {df2_clean['potential_avoidance'].sum()}")

Dataset 2 cleaning complete. Final shape: (2123, 29)
Key Investigation A variables created:
   - Predator pressure levels: {'none': 1584, 'minimal': 287, 'moderate': 184, 'high': 68}
   - Potential avoidance periods: 322


In [63]:
# Validate cleaned data
print("\n=== VALIDATION FOR INVESTIGATION A ===")

# Check for Investigation A critical variables
print("1. Validating Investigation A critical variables...")

# Dataset 1 critical variables for predator perception analysis
critical_vars_df1 = {
    'bat_landing_to_food': 'Avoidance behavior indicator',
    'risk': 'Risk-taking behavior',
    'reward': 'Foraging success',
    'rat_present': 'Predator presence',
    'behavior_type': 'Behavioral classification',
    'predator_avoidance_score': 'Composite avoidance measure'
}

for var, description in critical_vars_df1.items():
    if var in df1_clean.columns:
        missing = df1_clean[var].isnull().sum()
        if missing > 0:
            print(f"   WARNING: {missing} missing values in {var} ({description})")
        else:
            print(f"   ✓ {var} ({description}) - Complete")
    else:
        print(f"   ERROR: Missing critical variable {var} ({description})")

# Dataset 2 critical variables for population-level analysis
critical_vars_df2 = {
    'bat_landing_number': 'Bat activity level',
    'rat_minutes': 'Predator exposure duration',
    'food_availability': 'Resource competition',
    'predator_pressure': 'Predator pressure intensity',
    'potential_avoidance': 'Avoidance behavior indicator'
}

for var, description in critical_vars_df2.items():
    if var in df2_clean.columns:
        missing = df2_clean[var].isnull().sum()
        if missing > 0:
            print(f"   WARNING: {missing} missing values in {var} ({description})")
        else:
            print(f"   ✓ {var} ({description}) - Complete")
    else:
        print(f"   ERROR: Missing critical variable {var} ({description})")



=== VALIDATION FOR INVESTIGATION A ===
1. Validating Investigation A critical variables...
   ✓ bat_landing_to_food (Avoidance behavior indicator) - Complete
   ✓ risk (Risk-taking behavior) - Complete
   ✓ reward (Foraging success) - Complete
   ✓ rat_present (Predator presence) - Complete
   ✓ behavior_type (Behavioral classification) - Complete
   ✓ predator_avoidance_score (Composite avoidance measure) - Complete
   ✓ bat_landing_number (Bat activity level) - Complete
   ✓ rat_minutes (Predator exposure duration) - Complete
   ✓ food_availability (Resource competition) - Complete
   ✓ predator_pressure (Predator pressure intensity) - Complete
   ✓ potential_avoidance (Avoidance behavior indicator) - Complete


In [64]:
# 2. Validate Investigation A hypotheses testing readiness
print("\n2. Validating readiness for hypothesis testing...")

# Check for sufficient data in key conditions
if 'rat_present' in df1_clean.columns:
    rat_present_count = df1_clean['rat_present'].sum()
    rat_absent_count = (df1_clean['rat_present'] == 0).sum()
    print(f"   Dataset 1 - Rat present: {rat_present_count}, Rat absent: {rat_absent_count}")
    
    if rat_present_count < 30 or rat_absent_count < 30:
        print("   WARNING: Low sample sizes may affect statistical power")

if 'rat_present' in df2_clean.columns:
    rat_periods = df2_clean['rat_present'].sum()
    no_rat_periods = (df2_clean['rat_present'] == 0).sum()
    print(f"   Dataset 2 - Periods with rats: {rat_periods}, Periods without: {no_rat_periods}")



2. Validating readiness for hypothesis testing...
   Dataset 1 - Rat present: 311, Rat absent: 596
   Dataset 2 - Periods with rats: 545, Periods without: 1578


In [65]:
# 3. Check for Investigation A key behavioral patterns
print("\n3. Investigation A behavioral pattern validation...")

if 'avoidance_level' in df1_clean.columns:
    avoidance_dist = df1_clean['avoidance_level'].value_counts()
    print(f"   Avoidance behavior distribution: {avoidance_dist.to_dict()}")

if 'interaction_type' in df1_clean.columns:
    interaction_dist = df1_clean['interaction_type'].value_counts()
    print(f"   Interaction types: {interaction_dist.to_dict()}")

if 'predator_pressure' in df2_clean.columns:
    pressure_dist = df2_clean['predator_pressure'].value_counts()
    print(f"   Predator pressure distribution: {pressure_dist.to_dict()}")



3. Investigation A behavioral pattern validation...
   Avoidance behavior distribution: {'quick': 276, 'immediate': 263, 'delayed': 189, 'highly_delayed': 179}
   Interaction types: {'no_rat': 596, 'predation_fear': 271, 'competition': 40}
   Predator pressure distribution: {'none': 1584, 'minimal': 287, 'moderate': 184, 'high': 68}


In [66]:
# 4. Data quality summary
print(f"\n4. Data quality summary:")
print(f"   Dataset 1: {df1_clean.shape[0]} bat landings, {df1_clean.shape[1]} variables")
print(f"   Dataset 2: {df2_clean.shape[0]} observation periods, {df2_clean.shape[1]} variables")



4. Data quality summary:
   Dataset 1: 907 bat landings, 29 variables
   Dataset 2: 2123 observation periods, 29 variables


In [67]:
# 5. Investigation A readiness assessment
readiness_score = 0
max_score = 5

if 'bat_landing_to_food' in df1_clean.columns and df1_clean['bat_landing_to_food'].notna().sum() > 100:
    readiness_score += 1
    print("   ✓ Sufficient avoidance behavior data")
    
if 'rat_present' in df1_clean.columns and df1_clean['rat_present'].sum() > 50:
    readiness_score += 1
    print("   ✓ Sufficient rat encounter data")
    
if 'predator_avoidance_score' in df1_clean.columns:
    readiness_score += 1
    print("   ✓ Composite avoidance measure available")
    
if 'potential_avoidance' in df2_clean.columns:
    readiness_score += 1
    print("   ✓ Population-level avoidance indicators available")
    
if all(col in df1_clean.columns for col in ['risk', 'reward']):
    readiness_score += 1
    print("   ✓ Risk-reward behavioral data complete")
    
print(f"\n   Investigation A Readiness Score: {readiness_score}/{max_score}")
if readiness_score >= 4:
    print("   ✅ READY for Investigation A analysis!")
else:
    print("   ⚠️  Some data quality issues may affect analysis")


   ✓ Sufficient avoidance behavior data
   ✓ Sufficient rat encounter data
   ✓ Composite avoidance measure available
   ✓ Population-level avoidance indicators available
   ✓ Risk-reward behavioral data complete

   Investigation A Readiness Score: 5/5
   ✅ READY for Investigation A analysis!


In [68]:
# Save cleaned data
print("\n=== SAVING CLEANED DATA ===")

# Save cleaned datasets
df1_clean.to_csv("cleaned_data/dataset1_cleaned.csv", index=False)
df2_clean.to_csv("cleaned_data/dataset2_cleaned.csv", index=False)

print("Cleaned datasets saved to:")
print("   - cleaned_data/dataset1_cleaned.csv")
print("   - cleaned_data/dataset2_cleaned.csv")

# Save summary report
print("\n=== CLEANING SUMMARY REPORT ===")
report = []
report.append("=" * 60)
report.append("DATA CLEANING SUMMARY REPORT")
report.append("=" * 60)
report.append(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
report.append("")

# Dataset 1 summary
report.append("DATASET 1 (Behavioral Data):")
report.append(f"  - Original observations: Unknown (cleaned version has {df1_clean.shape[0]})")
report.append(f"  - Features after cleaning: {df1_clean.shape[1]}")

if 'behavior_type' in df1_clean.columns:
    report.append("  - Behavior categories:")
    for behavior, count in df1_clean['behavior_type'].value_counts().items():
        report.append(f"    * {behavior}: {count}")

# Dataset 2 summary
report.append("")
report.append("DATASET 2 (Time Series Data):")
report.append(f"  - Original observations: Unknown (cleaned version has {df2_clean.shape[0]})")
report.append(f"  - Features after cleaning: {df2_clean.shape[1]}")

if 'food_level' in df2_clean.columns:
    report.append("  - Food availability levels:")
    for level, count in df2_clean['food_level'].value_counts().items():
        report.append(f"    * {level}: {count}")

# Cleaning operations
report.append("")
report.append("CLEANING OPERATIONS PERFORMED:")
report.append("  1. Datetime conversion and validation")
report.append("  2. Missing value imputation")
report.append("  3. Categorical data standardization")
report.append("  4. Outlier detection and flagging")
report.append("  5. Feature engineering (time-based, interactions)")
report.append("  6. Data validation and range checking")
report.append("  7. Binary encoding for machine learning readiness")

# Recommended next steps
report.append("")
report.append("RECOMMENDED NEXT STEPS:")
report.append("  1. Exploratory data analysis")
report.append("  2. Statistical testing for rat vs no-rat conditions")
report.append("  3. Time series analysis for patterns")
report.append("  4. Machine learning model development")

# Write report to file
summary_report = "\n".join(report)
with open("cleaned_data/cleaning_summary_report.txt", "w") as f:
    f.write(summary_report)

print("   - cleaned_data/cleaning_summary_report.txt")
print("\n" + "=" * 60)
print("DATA CLEANING PIPELINE COMPLETED SUCCESSFULLY!")
print("=" * 60)



=== SAVING CLEANED DATA ===
Cleaned datasets saved to:
   - cleaned_data/dataset1_cleaned.csv
   - cleaned_data/dataset2_cleaned.csv

=== CLEANING SUMMARY REPORT ===
   - cleaned_data/cleaning_summary_report.txt

DATA CLEANING PIPELINE COMPLETED SUCCESSFULLY!
